In [ ]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
from datetime import datetime
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID = "1_1_01_CURP_ESTRUCTURA"

# --- 2. LÓGICA PURA DE LA MÉTRICA (CORREGIDA) ---

def validar_curp_estructura(curp: str) -> bool:
    """
    Valida la estructura de una CURP según la métrica 1_1_01,
    incluyendo la validación de las 32 entidades federativas.
    """
    if not isinstance(curp, str):
        return False
        
    curp_regex = (
        "^[A-Z][AEIOU][A-Z]{2}"  # 4 letras, 2da es vocal
        "[0-9]{2}(0[1-9]|1[0-2])(0[1-9]|[12][0-9]|3[01])"  # Fecha aammdd
        "[HM]"  # Sexo H o M
        # Abreviación de Entidad Federativa (32 estados + Nacido Extranjero)
        "(AS|BC|BS|CC|CS|CH|CL|CM|DF|DG|GT|GR|HG|JC|MC|MN|MS|NT|NL|OC|PL|QT|QR|SP|SL|SR|TC|TS|TL|VZ|YN|ZS|NE)"
        "[B-DF-HJ-NP-TV-Z]{3}"  # 3 Consonantes
        "[A-Z0-9]"  # Alfanumérico
        "[0-9]$"  # Numérico
    )
    
    if re.match(curp_regex, curp):
        return True
    else:
        return False

# --- 3. EL FLUJO DEL WORKER (PROCESAMIENTO POR LOTES) ---
def procesar_todas_las_declaraciones():
    """
    Procesa TODAS las declaraciones de la colección de origen
    y guarda los resultados en la colección 'metricas'.
    """
    client = None
    
    # Contadores para el resumen final
    total_documentos = 0
    con_curp_cumple = 0
    con_curp_no_cumple = 0
    sin_curp = 0
    
    try:
        # --- A. CONECTAR A LA BASE DE DATOS ---
        print(f"Conectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        
        # Accedemos a las dos colecciones
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        print("¡Conexión exitosa!")
        print(f"Leyendo de: {SOURCE_COLLECTION_NAME}")
        print(f"Escribiendo en: {METRICS_COLLECTION_NAME}")
        print("--- INICIANDO PROCESAMIENTO POR LOTES ---")

        # --- B. BUSCAR TODOS LOS DOCUMENTOS ---
        # Usamos una proyección para traer solo los campos que necesitamos.
        # Esto es MUCHO más rápido que traer el documento completo.
        proyeccion = {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosGenerales.curp": 1
        }
        
        # Iteramos sobre todos los documentos
        for doc in source_collection.find({}, proyeccion):
            total_documentos += 1
            original_id = doc["_id"]

            # --- C. LEER DATOS Y EJECUTAR MÉTRICA (CON LÓGICA 'SIN_DATO') ---
            curp = None
            try:
                curp = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosGenerales", {}).get("curp")
            except AttributeError:
                curp = None 

            # Lógica para manejar todos los casos
            if curp and curp.strip() != "":
                if validar_curp_estructura(curp):
                    resultado_metrica = "CUMPLE"
                    con_curp_cumple += 1
                else:
                    resultado_metrica = "NO_CUMPLE"
                    con_curp_no_cumple += 1
            else:
                resultado_metrica = "SIN_DATO"
                sin_curp += 1

            # --- CONSERVA TU SALIDA DE CONSOLA ---
            print(f"\nProcesando Documento ID: {original_id}")
            print("--- RESULTADO DEL ANÁLISIS ---")
            print(f"  > CURP: '{curp}'")
            print(f"  > Resultado de la Métrica: {resultado_metrica}")
            print("---------------------------------")
            # --- FIN DE LA SALIDA DE CONSOLA ---

            # --- D. GUARDAR RESULTADO EN LA COLECCIÓN 'metricas' ---
            
            # El filtro busca por el _id original
            filtro = { "_id": original_id }
            
            # La actualización usa $set para añadir o actualizar el campo de la métrica
            # No incluimos fecha_analisis
            actualizacion = {
                "$set": {
                    METRIC_ID: resultado_metrica
                }
            }
            
            # Usamos UPSERT=TRUE:
            # - Si un documento con este _id YA EXISTE en 'metricas', lo actualiza.
            # - Si NO EXISTE, crea un nuevo documento con este _id.
            target_collection.update_one(filtro, actualizacion, upsert=True)

            print(f"¡Resultado guardado/actualizado en 'metricas' para el ID: {original_id}!")
            
            # Opcional: imprimir un punto por cada documento para ver el progreso
            # sys.stdout.write('.')
            # sys.stdout.flush()

        print("\n--- PROCESAMIENTO POR LOTES FINALIZADO ---")
        print("Resumen:")
        print(f"  > Documentos Totales Procesados: {total_documentos}")
        print(f"  > Métrica 'CUMPLE': {con_curp_cumple}")
        print(f"  > Métrica 'NO_CUMPLE': {con_curp_no_cumple}")
        print(f"  > Métrica 'SIN_DATO': {sin_curp}")

    except ConnectionFailure:
        print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e:
        print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        if client:
            client.close()
            print("Conexión cerrada.")

# --- 4. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    procesar_todas_las_declaraciones()

Conectando a MongoDB en sistema1...
¡Conexión exitosa!
Leyendo de: all_data_20251021
Escribiendo en: metricas
--- INICIANDO PROCESAMIENTO POR LOTES ---

Procesando Documento ID: 68f81b8800535f910a29f694
--- RESULTADO DEL ANÁLISIS ---
  > CURP: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f694!

Procesando Documento ID: 68f81b8800535f910a29f695
--- RESULTADO DEL ANÁLISIS ---
  > CURP: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f695!

Procesando Documento ID: 68f81b8800535f910a29f696
--- RESULTADO DEL ANÁLISIS ---
  > CURP: 'None'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f696!

Procesando Documento ID: 68f81b8800535f910a29f697
--- RESULTADO DEL ANÁLISIS ---
 

KeyboardInterrupt: 